# Load Depenency

In [82]:
#Cleaned up Crypto Example
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
# from openpyxl import Workbook
import openpyxl

In [83]:
#does some error handling


import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [84]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, max_studies=50):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&max_rnk={max_studies}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def get_study_fields(self, search_expr, fields, max_studies=50, min_rnk=1,fmt="csv"):
        """Returns study content for specified fields

        Retrieves information from the study fields endpoint, which acquires specified information
        from a large (max 1000) studies. To see a list of all possible fields, check the class'
        study_fields attribute.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            fields (list(str)): A list containing the desired information fields.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.
            fmt (str): A string indicating the output format, csv or json. Defaults to csv.

        Returns:
            Either a dict, if fmt='json', or a list of records (e.g. a list of lists), if fmt='csv.
            Both containing the maximum number of study fields queried using the specified search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 1000
            ValueError: One of the fields is not valid! Check the study_fields attribute
                for a list of valid ones.
            ValueError: Format argument has to be either 'csv' or 'json'
        """
        if max_studies > 1000 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 1000")
        elif not set(fields).issubset(self.study_fields):
            raise ValueError(
                "One of the fields is not valid! Check the study_fields attribute for a list of valid ones."
            )
        else:
            concat_fields = ",".join(fields)
            # req = f"study_fields?expr={search_expr}&max_rnk={max_studies}&fields={concat_fields}"
            req = f"study_fields?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&fields={concat_fields}"
            if fmt == "csv":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._CSV}"
                return csv_handler(url)

            elif fmt == "json":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
                return json_handler(url)

            else:
                raise ValueError("Format argument has to be either 'csv' or 'json'")

    def get_study_count(self, search_expr):
        """Returns study count for specified search expression

        Retrieves the count of studies matching the text entered in search_expr.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.

        Returns:
            An integer

        Raises:
            ValueError: The search expression cannot be blank.
        """
        if not set(search_expr):
            raise ValueError("The search expression cannot be blank.")
        else:
            req = f"study_fields?expr={search_expr}&max_rnk=1&fields=NCTId"
            url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
            returned_data = json_handler(url)
            study_count = returned_data["StudyFieldsResponse"]["NStudiesFound"]
            return study_count

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

# Load Data

https://clinicaltrials.gov/api/gui/ref/crosswalks

Disease 2 search for

This is for dymanic input of diease searching

In [85]:
# disease=input()
# print(disease)

In [86]:
disease = 'synuclein'

#disease = 'depression'

# disease = 'diabetes'

# disease = 'tacs'



hit mesh some day

Pull ONE

#Explore Data

In [87]:
#change this out to NAME

ct = ClinicalTrials()
infodf= (ct.get_full_studies(search_expr=disease, max_studies=5 ))


In [88]:
# type(infodf)

In [89]:
number = print(infodf['FullStudiesResponse']['NStudiesFound'])

114


In [90]:
count = ct.get_study_count(search_expr=disease)


In [91]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )





temp = pd.DataFrame()

temp2 = pd.DataFrame()
new_df = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType", "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase","StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BriefSummary","IsFDARegulatedDrug","IsFDARegulatedDevice"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    iter2 = iter + (i-1)
    print(str(i) + "-" + str(iter2) + " of " + str(count))
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df = pd.concat([new_df, temp2])
 


1-999 of 114


In [92]:
new_df.columns = ["Index","NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BriefSummary","IsFDARegulatedDrug","IsFDARegulatedDevice"]

In [93]:
# new_df
new_df.shape

(114, 19)

In [94]:
new_df.to_excel(r'	Alpha-synuclein_data_raw_part_a.xlsx',  engine='openpyxl', index = False)

In [95]:
new_df.head

<bound method NDFrame.head of     Index        NCTId                                          Condition  \
0       1  NCT04020198  Parkinson Disease|Multiple System Atrophy|Rapi...   
1       2  NCT01380899           Parkinson Disease|Parkinsonian Disorders   
2       3  NCT03156647           Parkinson Disease|Parkinsonian Disorders   
3       4  NCT04732741  Oral Cancer|Oral Lichen Planus|Oral Leukoplaki...   
4       5  NCT05067192  Parkinson Disease|Dementia With Lewy Bodies|Mu...   
..    ...          ...                                                ...   
109   110  NCT04691661                                  Parkinson Disease   
110   111  NCT03775538  Parkinson Disease|Movement Disorders|Neuro-Deg...   
111   112  NCT02408562                                Parkinson's Disease   
112   113  NCT04810104        Parkinson Disease|Mild Cognitive Impairment   
113   114  NCT03174886              Primary Progressive Nonfluent Aphasia   

                                         Offi

In [96]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df2 = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    iter2 = iter + (i-1)
    print(str(i) + "-" + str(iter2) + " of " + str(count))
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df2 = pd.concat([new_df2, temp2])
 


1-999 of 114


In [97]:
new_df2.columns = ["index", "NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice",  "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame",
"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"]

In [98]:
new_df2.shape

(114, 18)

##cleaning

In [99]:
new_df.to_excel(r'Alpha-synuclein_data_raw_part_b.xlsx',  engine='openpyxl', index = False)

In [100]:
# print(type(df1))

In [101]:
result = pd.merge(new_df, new_df2,  on='NCTId', how='left')

In [102]:
result.shape

(114, 36)

IMPORTANT - this just drops the second index column - monitor or watch for mismatch or if you change the number of elements that are pulled from the first api

In [103]:
result = result.drop(result.columns[[20]], axis=1)

In [104]:
result.shape

(114, 35)

Save file

In [105]:
# writer = pd.ExcelWriter('test.xlsx')
# # write dataframe to excel
# result.to_excel(writer)
# # save the excel
# writer.save()
# print('DataFrame is written successfully to Excel File.')

In [106]:
result.to_excel(r'Parkinsonian_data_raw.xlsx',  engine='openpyxl', index = False)



In [107]:
result.to_csv(r'Parkinsonian_data_raw.csv', index = False)

In [108]:
# result.to_hdf('data_raw.h5', key='df', mode='w')  